# Prise en main de Polars

Polars est un package Python permettant de manipuler les données tabulaires à partir de différents types de fichiers (CSV, Parquet, etc.). Il est une alternative directe et moderne à Pandas. Pour en savoir plus, la lecture de cet article du blog sur le site du SSPHub. > TODO lien

Pour bien débuter, on installe les packages nécessaires au fonctionnement de ce notebook et on importe toutes les fonctions à utiliser.


In [ ]:
!pip install polars pynsee[full] s3fs

In [ ]:
import os
import polars as pl
import s3fs
from pynsee.download import download_file

# Lecture de données

Les exemples fournis dans ce notebook utiliseront les données de la BPE (à l'instar du [module de découverte du tidyverse dans utilitr](https://www.book.utilitr.org/03_fiches_thematiques/fiche_tidyverse)).

On exploite ici deux possibilités :
1. charger les données via le module Python `pynsee`
2. charger depuis le dossier `donnees-insee` du datalab TODO


## Via Pynsee

In [ ]:
pandas_df_bpe = download_file("BPE_ENS") # pynsee renvoie un dataframe pandas
df = pl.from_pandas(pandas_df_bpe)
df.head(5)

## Via le stockage public du datalab

In [ ]:
# WIP need to deal with types for parquet
# Create filesystem object
S3_ENDPOINT_URL = "https://" + os.environ["AWS_S3_ENDPOINT"]
fs = s3fs.S3FileSystem(client_kwargs={'endpoint_url': S3_ENDPOINT_URL})
BUCKET = "donnees-insee/diffusion/BPE/2019"

with fs.open(f"{BUCKET}/BPE_ENS.csv") as bpe_csv:
    df_bpe = pl.read_csv(bpe_csv)
    print(df_bpe.head())
    #with fs.open(f"{BUCKET}/BPE_ENS.parquet", "w") as bpe_parquet:    
    df_bpe.write_parquet("bpe.parquet")

In [27]:
df.write_parquet("bpe.parquet")
df_bpe = pl.read_parquet("bpe.parquet")

# Comment utiliser Polars ?

A l'instar d'autres outils modernes d'exploitation des données, Polars expose un modèle de traitement basé sur des fonctions de haut niveau, comme `select`, `filter` ou `groupby`, qui empruntent au langage SQL une logique expressive du "quoi ?" plutôt que du "comment ?".

Dans l'exemple qui suit, on commence par déclarer une exécution retardée (via `lazy()`) qui va permettre au moteur sous-jacent d'optimiser le traitement complet. Puis on exprime à l'aide des fonctions de haut niveau ce que l'on veut faire :
1. filtrer le jeu de données pour ne garder les lignes pour lesquelles la colonne `TYPEQU` vaut `B316` (les stations-services)
2. on regroupe au niveau département
3. on compte le nombre d'occurrences pour chaque département via `agg`
4. le dernier appel - `collect()` - indique que le traitement peut être lancé (et donc optimisé, parallelisé par Polars).

In [28]:
df_stations_service = df_bpe.lazy().filter( # 1.
    pl.col("TYPEQU") == "B316"
).groupby( # 2.
    "DEP"
).agg( # 3.
    pl.count().alias("NB_STATION_SERVICE")
).collect() # 4.

df_stations_service.head(5)

DEP,NB_STATION_SERVICE
str,u32
"""34""",115
"""69""",158
"""48""",24
"""44""",139
"""79""",55


## Sélection de données

Deux types de sélections sont possibles :
1. une sélection de variables (en colonne), avec `select`
2. une sélection d'observations (en ligne), avec `filter`

La combinaison des deux se faisant en chaînant l'appel à ces deux fonctions.

### Sélection de variables

Commençons par sélectionner des variables en utilisant leurs noms :

In [30]:
df_bpe.select(
    ["DEPCOM", "TYPEQU", "NB_EQUIP"]
).head(5)

DEPCOM,TYPEQU,NB_EQUIP
str,str,str
"""01001""","""A129""","""1"""
"""01001""","""A401""","""2"""
"""01001""","""A402""","""1"""
"""01001""","""A404""","""2"""
"""01001""","""A405""","""2"""


Puis en utilisant leurs positions :

In [31]:
df_bpe[:, 1:5].head(5)

AN,BV2012,DEP,DEPCOM
str,str,str,str
"""2021""","""01093""","""01""","""01001"""
"""2021""","""01093""","""01""","""01001"""
"""2021""","""01093""","""01""","""01001"""
"""2021""","""01093""","""01""","""01001"""
"""2021""","""01093""","""01""","""01001"""


On peut également s'appuyer sur des motifs de sélection des noms de colonnes mobilisant des expressions régulières (ici `^DEP.*$` signifiant "débute par DEP"):

In [39]:
df_bpe.select(
    pl.col("^DEP.*$")
).head(5)

DEP,DEPCOM
str,str
"""01""","""01001"""
"""01""","""01001"""
"""01""","""01001"""
"""01""","""01001"""
"""01""","""01001"""


La fonction `select` acceptant des `list` Python, on peut construire des sélecteurs assez puissants :

In [44]:
dep_cols = [cols for cols in df_bpe.columns if cols.startswith("DEP")] 

df_bpe.select(dep_cols).head(5)

DEP,DEPCOM
str,str
"""01""","""01001"""
"""01""","""01001"""
"""01""","""01001"""
"""01""","""01001"""
"""01""","""01001"""


### Sélection d'observation

TODO

## Renommage de variables

La fonction `rename` permet de lister les colonnes à renommer via un dictionnaire Python :

In [47]:
df_bpe.rename({
    "DEPCOM" : "code_commune"
}).head(5)

AAV2020,AN,BV2012,DEP,code_commune,DOM,EPCI,DCIRIS,REG,SDOM,TYPEQU,UU2020,NB_EQUIP
str,str,str,str,str,str,str,str,str,str,str,str,str
"""524""","""2021""","""01093""","""01""","""01001""","""A""","""200069193""","""010010000""","""84""","""A1""","""A129""","""CSZ""","""1"""
"""524""","""2021""","""01093""","""01""","""01001""","""A""","""200069193""","""010010000""","""84""","""A4""","""A401""","""CSZ""","""2"""
"""524""","""2021""","""01093""","""01""","""01001""","""A""","""200069193""","""010010000""","""84""","""A4""","""A402""","""CSZ""","""1"""
"""524""","""2021""","""01093""","""01""","""01001""","""A""","""200069193""","""010010000""","""84""","""A4""","""A404""","""CSZ""","""2"""
"""524""","""2021""","""01093""","""01""","""01001""","""A""","""200069193""","""010010000""","""84""","""A4""","""A405""","""CSZ""","""2"""


_Fun fact_, la logique est l'inverse de celle de dplyr. :)

Comme vu plus haut, construire des expressions de renommage plus complexes pourra se faire en pur Python :

In [56]:
cols_minuscules = {cols:cols.lower() for cols in df_bpe.columns}

df_bpe.rename(cols_minuscules).head(5)

aav2020,an,bv2012,dep,depcom,dom,epci,dciris,reg,sdom,typequ,uu2020,nb_equip
str,str,str,str,str,str,str,str,str,str,str,str,str
"""524""","""2021""","""01093""","""01""","""01001""","""A""","""200069193""","""010010000""","""84""","""A1""","""A129""","""CSZ""","""1"""
"""524""","""2021""","""01093""","""01""","""01001""","""A""","""200069193""","""010010000""","""84""","""A4""","""A401""","""CSZ""","""2"""
"""524""","""2021""","""01093""","""01""","""01001""","""A""","""200069193""","""010010000""","""84""","""A4""","""A402""","""CSZ""","""1"""
"""524""","""2021""","""01093""","""01""","""01001""","""A""","""200069193""","""010010000""","""84""","""A4""","""A404""","""CSZ""","""2"""
"""524""","""2021""","""01093""","""01""","""01001""","""A""","""200069193""","""010010000""","""84""","""A4""","""A405""","""CSZ""","""2"""
